# Resource Monitoring and Troubleshooting - Practice Notebook

**Prerequisites**: Python 3.10+, PyTorch 2.6.0+, CUDA 12.4+

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Monitor GPU Resources**: Track memory usage and utilization in real-time
2. **Detect Memory Leaks**: Identify and fix common PyTorch memory issues
3. **Monitor System Resources**: Track CPU, RAM, and disk usage
4. **Implement Retry Strategies**: Build robust error handling with exponential backoff
5. **Profile Performance**: Use PyTorch Profiler to find bottlenecks
6. **Visualize Metrics**: Create real-time monitoring dashboards

## Why Resource Monitoring Matters in AI/ML

Resource monitoring is crucial for:

- **Performance Optimization**: GPU resources are expensive. Identifying bottlenecks helps you train faster and cheaper.
- **Cost Management**: Cloud GPU instances can cost $1-10+ per hour. Efficient resource use saves money.
- **Reliability**: Out-of-memory errors can crash training jobs that took hours. Prevention is key.
- **Debugging**: Understanding resource usage patterns helps diagnose mysterious failures.
- **Production Readiness**: Production ML systems need monitoring to ensure SLAs and detect issues.

## How This Notebook Works

Each exercise follows this pattern:
1. **Introduction**: Explains the concept and why it matters
2. **Exercise**: A TODO section for you to try implementing
3. **Solution**: Complete working code with detailed comments
4. **Key Insights**: Important takeaways from the exercise

💡 **Tip**: Try completing each TODO before looking at the solution\!

## Setup and Environment Verification

**Purpose**: Import essential libraries and verify your environment is properly configured.

**Key Libraries**:
- **torch**: PyTorch deep learning framework with CUDA support
- **psutil**: Cross-platform library for system and process monitoring
- **matplotlib**: For creating visualizations of resource usage
- **numpy**: Numerical computing (used for data manipulation)

**What to Check**:
- Python version (should be 3.10+)
- PyTorch version (should be 2.6.0+)
- CUDA availability (True if you have a GPU)

If CUDA shows False but you have a GPU, check:
1. NVIDIA drivers are installed
2. PyTorch was installed with CUDA support
3. CUDA version matches PyTorch requirements

In [ ]:
import sys, torch, psutil, time
import numpy as np
import matplotlib.pyplot as plt
from functools import wraps
import random

print(f'Python: {sys.version}')
print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.cuda.is_available()}')

## Exercise 1: GPU Monitoring

**Purpose**: Learn to monitor GPU resources programmatically using PyTorch's CUDA API.

### Why GPU Monitoring Matters

GPUs are the most expensive resource in deep learning:
- A single NVIDIA A100 GPU costs $10,000-15,000
- Cloud GPU instances: $1-10+ per hour
- Training large models can take weeks of GPU time

Monitoring helps you:
- **Detect underutilization**: Are you using 10% of GPU when you could use 80%?
- **Prevent OOM errors**: Know when you're approaching memory limits
- **Optimize batch sizes**: Find the sweet spot between speed and memory
- **Debug memory leaks**: Track memory growth over time

### What You'll Monitor

1. **Device Name**: Which GPU model you're using (e.g., RTX 3090, A100)
2. **Total Memory**: Maximum GPU memory available (e.g., 24GB, 40GB)
3. **Allocated Memory**: Memory currently in use by PyTorch tensors
4. **Reserved Memory**: Memory reserved by PyTorch's caching allocator
5. **Free Memory**: Available memory for new allocations

### Key Concepts

- **Allocated vs Reserved**: PyTorch reserves memory in chunks for efficiency. Allocated is what's actually used.
- **Memory Fragmentation**: Even with free memory, large allocations might fail due to fragmentation.
- **Multiple GPUs**: If you have multiple GPUs, monitor each one separately.

### Your Task

Implement `monitor_gpu()` function that:
1. Checks if CUDA is available
2. Iterates through all available GPUs
3. Displays memory statistics for each GPU

**Hints**:
- Use `torch.cuda.is_available()` to check for CUDA
- Use `torch.cuda.device_count()` for number of GPUs
- Use `torch.cuda.get_device_properties(i).total_memory` for total memory
- Use `torch.cuda.memory_allocated(i)` for used memory

In [ ]:
# TODO: Implement GPU monitoring
def monitor_gpu():
    if not torch.cuda.is_available():
        print('CUDA not available')
        return
    # Your code here
    pass

monitor_gpu()

### Solution:

In [ ]:
def monitor_gpu():
    if not torch.cuda.is_available():
        print('CUDA not available')
        return
    
    for i in range(torch.cuda.device_count()):
        print(f'GPU {i}: {torch.cuda.get_device_name(i)}')
        total = torch.cuda.get_device_properties(i).total_memory / 1024**3
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        reserved = torch.cuda.memory_reserved(i) / 1024**3
        print(f'  Total: {total:.2f} GB')
        print(f'  Allocated: {allocated:.2f} GB')
        print(f'  Reserved: {reserved:.2f} GB')

monitor_gpu()

## Exercise 2: Detecting and Fixing Memory Leaks

**Purpose**: Learn to identify and fix the most common memory leak in PyTorch code.

### The Problem: Hidden Computation Graphs

PyTorch uses **automatic differentiation** (autograd) which builds a computation graph:
- Every tensor operation is recorded
- Gradients are computed by traversing this graph backward
- The graph is stored in memory until explicitly freed

**Common Mistake**: Storing tensors that still have computation graphs attached:

```python
losses = []
for epoch in range(100):
    loss = model(data).sum()
    losses.append(loss)  # LEAK\! Stores entire graph
```

### Why This Causes Memory Leaks

When you append a tensor with `requires_grad=True` to a list:
1. The tensor keeps a reference to the computation graph
2. The graph includes all intermediate tensors and operations
3. These never get garbage collected
4. Memory usage grows linearly with training steps

**Real Impact**: A model training for 10,000 steps might leak 10GB+ of memory\!

### The Solution: Detach from Graph

Use `.item()` to extract the scalar value:

```python
losses.append(loss.item())  # FIX\! Only stores the number
```

Alternatives:
- `loss.detach()` - detaches but keeps as tensor
- `loss.cpu().numpy()` - converts to numpy array
- Don't store it at all if not needed

### Your Task

1. Run `train_with_leak()` and observe memory growth
2. Identify the line causing the leak
3. Fix it in `train_fixed()`
4. Verify memory stays constant

**What to Observe**:
- Memory should grow steadily in the leaky version
- Memory should stabilize in the fixed version
- The difference can be dramatic (100s of MB)

In [ ]:
# TODO: Fix the memory leak
def train_with_leak():
    model = torch.nn.Linear(1000,1000).cuda() if torch.cuda.is_available() else torch.nn.Linear(1000,1000)
    optimizer = torch.optim.Adam(model.parameters())
    losses = []  # Memory leak\!
    
    for i in range(50):
        x = torch.randn(32,1000).cuda() if torch.cuda.is_available() else torch.randn(32,1000)
        y = model(x).sum()
        optimizer.zero_grad()
        y.backward()
        optimizer.step()
        losses.append(y)  # LEAK: stores entire graph
        
        if i%10==0 and torch.cuda.is_available():
            print(f'Step {i}: {torch.cuda.memory_allocated()/1024**2:.1f} MB')

train_with_leak()

### Solution:

In [ ]:
def train_fixed():
    model = torch.nn.Linear(1000,1000).cuda() if torch.cuda.is_available() else torch.nn.Linear(1000,1000)
    optimizer = torch.optim.Adam(model.parameters())
    losses = []
    
    for i in range(50):
        x = torch.randn(32,1000).cuda() if torch.cuda.is_available() else torch.randn(32,1000)
        y = model(x).sum()
        optimizer.zero_grad()
        y.backward()
        optimizer.step()
        losses.append(y.item())  # FIX: use .item() to get scalar
        
        if i%10==0 and torch.cuda.is_available():
            print(f'Step {i}: {torch.cuda.memory_allocated()/1024**2:.1f} MB')

if torch.cuda.is_available():
    torch.cuda.empty_cache()
train_fixed()

## Exercise 3: System Resource Monitoring

**Purpose**: Monitor CPU, RAM, and disk usage to ensure your entire system is healthy.

### Beyond GPU: Why Monitor System Resources?

While GPU is critical for training, bottlenecks can occur elsewhere:

**CPU Bottlenecks**:
- Data loading and preprocessing
- DataLoader workers processing images/text
- Augmentation pipelines
- If CPU is at 100%, your GPU might be idle waiting for data\!

**RAM Issues**:
- Large datasets need to fit in RAM
- Running out of RAM causes swapping (very slow)
- Multiple processes competing for memory
- System might kill your process (OOM killer)

**Disk Space**:
- Checkpoints can be huge (10GB+ for large models)
- Logs and cached datasets accumulate
- Running out of disk crashes training

### Understanding the Metrics

**CPU Usage**:
- 0-100% per core (or overall)
- High usage during data loading is normal
- Sustained 100% might indicate bottleneck

**Memory**:
- Total: Physical RAM installed
- Used: Currently allocated memory
- Available: Free + reclaimable cache
- Percent: Used / Total

**Disk**:
- Total/Used/Free in bytes
- Percent full
- Watch for >90% usage (danger zone)

### The psutil Library

`psutil` (Python system and process utilities) provides:
- Cross-platform compatibility (Linux, Windows, macOS)
- Real-time monitoring
- Per-process and system-wide stats
- Easy-to-use API

### Your Task

Implement `monitor_system()` to display:
1. CPU usage percentage and core count
2. RAM total, used, and available (in GB)
3. Disk total, used, and free (in GB)

**Hints**:
- `psutil.cpu_percent(interval=1)` - CPU usage (wait 1 sec for accurate reading)
- `psutil.virtual_memory()` - RAM stats
- `psutil.disk_usage('/')` - Disk stats for root partition
- Convert bytes to GB: `bytes / 1024**3`

In [ ]:
# TODO: Implement system monitoring
def monitor_system():
    # Monitor CPU, RAM, disk
    pass

monitor_system()

### Solution:

In [ ]:
def monitor_system():
    print('='*50)
    print('System Resources')
    print('='*50)
    
    # CPU
    print(f'\nCPU Usage: {psutil.cpu_percent(interval=1)}%')
    print(f'CPU Cores: {psutil.cpu_count()}')
    
    # Memory
    mem = psutil.virtual_memory()
    print(f'\nTotal RAM: {mem.total/1024**3:.1f} GB')
    print(f'Used RAM: {mem.used/1024**3:.1f} GB ({mem.percent}%)')
    print(f'Available RAM: {mem.available/1024**3:.1f} GB')
    
    # Disk
    disk = psutil.disk_usage('/')
    print(f'\nTotal Disk: {disk.total/1024**3:.1f} GB')
    print(f'Used Disk: {disk.used/1024**3:.1f} GB ({disk.percent}%)')
    print(f'Free Disk: {disk.free/1024**3:.1f} GB')

monitor_system()

## Exercise 4: Retry Logic with Exponential Backoff

**Purpose**: Build robust error handling for unreliable operations (network, APIs, remote storage).

### The Problem: Transient Failures

In production ML systems, failures happen:
- **Network timeouts**: Downloading models from Hugging Face
- **API rate limits**: Calling external APIs (OpenAI, cloud services)
- **Temporary overloads**: Remote storage systems under stress
- **Cloud instability**: Brief service interruptions

Without retry logic, a single transient error crashes your entire training job.

### Naive Retry (DON'T DO THIS)

```python
for i in range(5):
    try:
        return download_model()
    except:
        time.sleep(1)  # Wait same time every retry
```

**Problems**:
- Fixed delay doesn't adapt to problem severity
- Can overwhelm already-struggling services
- No jitter (synchronized retries from multiple clients)

### Exponential Backoff: The Right Way

**Pattern**: Wait longer after each failure

```
Attempt 1: Wait 1 second
Attempt 2: Wait 2 seconds
Attempt 3: Wait 4 seconds
Attempt 4: Wait 8 seconds
Attempt 5: Wait 16 seconds
```

**Benefits**:
1. **Self-healing**: Gives services time to recover
2. **Adaptive**: Longer waits for persistent problems
3. **Prevents cascade failures**: Doesn't overwhelm struggling services

### Adding Jitter

Jitter = small random delay added to prevent synchronized retries:

```python
delay = base_delay * (2 ** attempt)
jitter = random.uniform(0, delay * 0.1)  # 10% jitter
actual_delay = delay + jitter
```

**Why jitter matters**: If 1000 clients all retry at exact same time, they create a thundering herd.

### Real-World Examples

- **AWS APIs**: Use exponential backoff (official recommendation)
- **Google Cloud**: Built-in retry with backoff
- **Hugging Face Hub**: Retries downloads automatically
- **HTTP libraries**: requests, httpx support retry strategies

### Your Task

Implement a decorator that:
1. Catches exceptions from wrapped function
2. Retries with exponential backoff
3. Adds random jitter
4. Gives up after max_retries

**Parameters to implement**:
- `max_retries`: Maximum number of attempts
- `base_delay`: Initial wait time (e.g., 1 second)
- `exponential_base`: Multiplier for each retry (typically 2)

**Test it** with `unreliable_function()` that fails randomly 70% of the time.

In [ ]:
# TODO: Implement retry decorator
def retry_with_backoff(max_retries=3, base_delay=1):
    def decorator(func):
        def wrapper(*args, **kwargs):
            # Your code here
            pass
        return wrapper
    return decorator

@retry_with_backoff(max_retries=5)
def unreliable_function():
    if random.random() < 0.7:
        raise ConnectionError('Random failure')
    return 'Success\!'

try:
    result = unreliable_function()
    print(result)
except Exception as e:
    print(f'Failed: {e}')

### Solution:

In [ ]:
def retry_with_backoff(max_retries=3, base_delay=1, exp_base=2):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_retries - 1:
                        raise
                    delay = min(base_delay * (exp_base ** attempt), 60)
                    jitter = random.uniform(0, delay * 0.1)
                    sleep_time = delay + jitter
                    print(f'Attempt {attempt+1} failed: {e}')
                    print(f'Retrying in {sleep_time:.2f}s...')
                    time.sleep(sleep_time)
        return wrapper
    return decorator

@retry_with_backoff(max_retries=5, base_delay=0.5)
def unreliable_function():
    if random.random() < 0.7:
        raise ConnectionError('Random failure')
    return 'Success\!'

try:
    result = unreliable_function()
    print(f'\n✅ {result}')
except Exception as e:
    print(f'\n❌ Failed: {e}')

## Exercise 5: Performance Profiling with PyTorch Profiler

**Purpose**: Use PyTorch Profiler to identify performance bottlenecks and optimize your code.

### The Performance Mystery

Your training is slow. But why?
- Is it the model forward pass?
- Is it the backward pass (gradients)?
- Is it the optimizer step?
- Is it data transfer between CPU and GPU?
- Is it a specific layer (attention, convolution)?

**Guessing is wasteful**. Profiling gives you data.

### What PyTorch Profiler Measures

**CPU Time**:
- Python overhead
- Data preprocessing
- Kernel launches
- Memory allocations

**CUDA Time** (GPU):
- Actual GPU kernel execution
- Memory transfers (CPU ↔ GPU)
- Synchronization overhead

**Memory**:
- Allocations and deallocations
- Peak memory usage
- Memory bandwidth

### Reading Profiler Output

The profiler shows a table like:

```
Name                    CPU Time    CUDA Time    Calls
aten::linear            10.5ms      50.2ms       100
aten::relu              2.1ms       5.3ms        50
aten::backward          5.2ms       25.1ms       50
```

**How to interpret**:
- **High CPU time**: Python overhead, data loading
- **High CUDA time**: GPU computation bottleneck
- **Many calls**: Might benefit from fusion
- **CPU > CUDA**: GPU is idle, CPU is bottleneck

### Common Bottlenecks and Solutions

**1. Data Loading Bottleneck**
- Symptom: GPU utilization < 50%, high CPU time
- Solution: More DataLoader workers, prefetch, faster disk

**2. Small Batch Size**
- Symptom: Low GPU utilization, many kernel launches
- Solution: Increase batch size, gradient accumulation

**3. CPU-GPU Transfers**
- Symptom: High time in memory copy operations
- Solution: Pin memory, avoid unnecessary .cpu()/.cuda() calls

**4. Slow Operations**
- Symptom: One operation dominates CUDA time
- Solution: Optimize that operation, use fused kernels, consider alternatives

### Profiling Best Practices

1. **Warm up**: Run a few iterations before profiling (CUDA initialization overhead)
2. **Profile production workload**: Use real batch sizes and model architecture
3. **Focus on hot spots**: Optimize the top 3-5 operations first (80/20 rule)
4. **Profile iteratively**: Profile → Optimize → Profile again

### Your Task

This exercise profiles a simple neural network:
1. Creates a 3-layer model
2. Runs 10 forward and backward passes
3. Collects CPU and CUDA timing
4. Displays the top 10 operations

**What to look for**:
- Which operations take the most time?
- Is time spent on forward pass or backward pass?
- Are there unexpected slow operations?

**Extension ideas**:
- Export trace to Chrome: `prof.export_chrome_trace('trace.json')`
- View in chrome://tracing for visual timeline
- Compare with different batch sizes
- Profile your own models

In [ ]:
import torch.profiler as profiler

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.nn.Sequential(
    torch.nn.Linear(1000, 500),
    torch.nn.ReLU(),
    torch.nn.Linear(500, 10)
).to(device)

data = torch.randn(32, 1000).to(device)

activities = [profiler.ProfilerActivity.CPU]
if torch.cuda.is_available():
    activities.append(profiler.ProfilerActivity.CUDA)

with profiler.profile(activities=activities, record_shapes=True) as prof:
    for _ in range(10):
        output = model(data)
        loss = output.sum()
        loss.backward()

print('\nTop 10 operations by CPU time:')
print(prof.key_averages().table(sort_by='cpu_time_total', row_limit=10))

## Exercise 6: Real-Time Monitoring Dashboard

**Purpose**: Visualize resource usage and training metrics in real-time to understand system behavior.

### Why Visualize During Training?

Numbers in logs are hard to interpret:
```
Step 100: Loss=2.3, GPU=4521MB, CPU=45%
Step 200: Loss=2.1, GPU=4523MB, CPU=47%
Step 300: Loss=1.9, GPU=4890MB, CPU=44%
```

**Questions you can't easily answer**:
- Is memory growing steadily (leak) or stable?
- Are there sudden spikes?
- Is loss decreasing smoothly or jumping around?
- What's the relationship between resource usage and training progress?

**Visualization makes patterns obvious**.

### What to Monitor During Training

**GPU Memory**:
- Should stabilize after a few steps
- Steady growth = memory leak
- Spikes = large batches or special operations
- Plateaus = OOM about to happen

**CPU Usage**:
- High during data loading
- Should correlate with training steps
- Sustained 100% = bottleneck
- Near 0% = GPU training, no data loading

**Training Loss**:
- Should decrease over time
- Smooth curve = good learning rate
- Spiky = learning rate too high or batch size too small
- Flat = learning stopped (bad)

### Real-World Monitoring Tools

In production, use specialized tools:

**Tensorboard**:
```python
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
writer.add_scalar('Loss/train', loss, step)
writer.add_scalar('GPU/memory', gpu_mem, step)
```

**Weights & Biases (wandb)**:
```python
import wandb
wandb.init(project='my-project')
wandb.log({'loss': loss, 'gpu_memory': gpu_mem})
```

**MLflow**:
```python
import mlflow
mlflow.log_metric('loss', loss, step=step)
```

### Interpreting the Plots

**Healthy Training**:
- GPU memory: Flat after warmup
- CPU usage: Moderate, consistent
- Loss: Smooth decrease

**Memory Leak**:
- GPU memory: Linear increase
- Eventually: OOM crash

**CPU Bottleneck**:
- CPU usage: 100%
- GPU memory: Stable but low
- Training: Slower than expected

**Learning Issues**:
- Loss: Not decreasing
- Loss: Exploding (NaN)
- Loss: Very spiky

### Your Task

This exercise:
1. Trains a simple model for 30 steps
2. Collects GPU memory, CPU usage, and loss at each step
3. Creates three side-by-side plots
4. Uses matplotlib for visualization

**What to observe**:
- GPU memory should stabilize quickly
- CPU usage will vary (data loading + training)
- Loss should decrease (though it's random data, so might not)

**Try experimenting**:
- Change `num_steps` to see longer trends
- Introduce a memory leak (remove `.item()`) and watch GPU plot
- Use larger batch size and see GPU memory increase
- Add more DataLoader workers and watch CPU usage

### Production Tip

For long training runs:
- Log metrics to files (CSV, JSON)
- Use proper logging frameworks (TensorBoard, wandb)
- Set up alerts for anomalies (sudden memory spike, loss explosion)
- Monitor remotely (web dashboards)
- Save checkpoints regularly based on metrics

In [ ]:
def monitor_training(num_steps=30):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = torch.nn.Sequential(
        torch.nn.Linear(1000, 500),
        torch.nn.ReLU(),
        torch.nn.Linear(500, 10)
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters())
    
    gpu_mem, cpu_usage, losses = [], [], []
    
    for step in range(num_steps):
        data = torch.randn(32, 1000).to(device)
        target = torch.randint(0, 10, (32,)).to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = torch.nn.functional.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if torch.cuda.is_available():
            gpu_mem.append(torch.cuda.memory_allocated() / 1024**2)
        cpu_usage.append(psutil.cpu_percent())
        losses.append(loss.item())
    
    # Plot
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    if gpu_mem:
        axes[0].plot(gpu_mem, 'b-')
        axes[0].set_title('GPU Memory (MB)')
        axes[0].set_xlabel('Step')
        axes[0].grid(True)
    
    axes[1].plot(cpu_usage, 'g-')
    axes[1].set_title('CPU Usage (%)')
    axes[1].set_xlabel('Step')
    axes[1].grid(True)
    
    axes[2].plot(losses, 'r-')
    axes[2].set_title('Training Loss')
    axes[2].set_xlabel('Step')
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()

monitor_training()

## Summary

### What You Practiced:

1. ✅ **GPU Monitoring**: Track memory and utilization
2. ✅ **Memory Leaks**: Detect and fix PyTorch memory issues
3. ✅ **System Resources**: Monitor CPU, RAM, disk
4. ✅ **Retry Logic**: Implement exponential backoff
5. ✅ **Profiling**: Identify performance bottlenecks
6. ✅ **Visualization**: Real-time monitoring dashboard

### Key Takeaways:

- Use `torch.cuda.memory_allocated()` to track GPU memory
- Always use `.item()` to detach scalars from computation graph
- `psutil` provides cross-platform system monitoring
- Exponential backoff prevents overwhelming failing services
- PyTorch Profiler reveals performance bottlenecks
- Visualize metrics to understand system behavior

### Next Steps:

1. Apply these techniques to your own projects
2. Set up monitoring in production environments
3. Create alerts for resource thresholds
4. Review [01_resource_monitoring.md](./01_resource_monitoring.md) for more details
5. Continue to [02_dockerization_practice.ipynb](./02_dockerization_practice.ipynb)

### Useful Functions:

```python
# GPU monitoring
torch.cuda.memory_allocated()
torch.cuda.memory_reserved()
torch.cuda.empty_cache()

# System monitoring
psutil.cpu_percent()
psutil.virtual_memory()
psutil.disk_usage('/')

# Profiling
torch.profiler.profile()
```